## Setting elliptic curve parameter

In [40]:
from umbral import config
from umbral.curve import SECP256K1
config.set_default_curve(SECP256K1)

UmbralConfigurationError: You can only set the default curve once.  Do it once and then leave it alone.

## Generating Key-Pair for Alice

In [68]:
from umbral import keys, signing
alices_private_key = keys.UmbralPrivateKey.gen_key()
alices_public_key = alices_private_key.get_pubkey()

# alices_signing_key = keys.UmbralPrivateKey.gen_key()
# alices_verifying_key = alices_signing_key.get_pubkey()
alices_signer = signing.Signer(private_key=alices_private_key)

## Common methods for reading-writing a file

In [69]:
plainTextFile = "alice/secretFile.txt"
cipherTextFile = "IPFS/encSecretFile.txt"
decryptedTestFile = "bob/secretFile.txt"
# function to read a file into bytes 
def readFileIntoBytes(filePath):
    """This method reads the file data into bytes"""
    with open(filePath, 'rb') as myfile:
        return myfile.read()

In [70]:
# function to write bytes into a file
def writeBytesIntoFile(filePath, bytesToWrite):
    """This method to write bytes into a file"""
    with open(filePath, 'wb') as myfile:
        myfile.write(bytesToWrite)

## Encrypting with a Alice's public key

In [71]:
from umbral import pre
plaintext = readFileIntoBytes(plainTextFile)
ciphertext, capsule = pre.encrypt(alices_public_key, plaintext)
writeBytesIntoFile(cipherTextFile, ciphertext)
# capsule and ciphertext are stored in IPFS

## Generating Key-Pair for Bob

In [72]:
bobs_private_key = keys.UmbralPrivateKey.gen_key()
bobs_public_key = bobs_private_key.get_pubkey()

# Now Bob wants access of that text from Alice

## Generating 20 Key Frags by Alice

When Alice wants to grant Bob access to open her encrypted messages, she creates threshold split re-encryption keys, or “kfrags”, which are next sent to N proxies or Ursulas.

In [73]:
kfrags = pre.generate_kfrags(delegating_privkey=alices_private_key,
                              signer=alices_signer,
                              receiving_pubkey=bobs_public_key,
                              threshold=10,
                              N=20)
kfrags

[KFrag:aae0081e1bd4f1a,
 KFrag:4e4950746c67967,
 KFrag:27f8420a8254f93,
 KFrag:f49ec6196a0218d,
 KFrag:0b472c02567909f,
 KFrag:17e18735841d638,
 KFrag:23bbdfa98dfa9d4,
 KFrag:9972938a48bb948,
 KFrag:0099cb38151d3c4,
 KFrag:e2e7aba175b6194,
 KFrag:42997a9e6117651,
 KFrag:0f96871d6bbe240,
 KFrag:9f9af153fd3f89a,
 KFrag:448dc37ae22faff,
 KFrag:9e8e8eef981adb2,
 KFrag:864159151033678,
 KFrag:49686957b10b2db,
 KFrag:8d0f411f543e89b,
 KFrag:2bd1915510c701a,
 KFrag:49107a749fa3e5c]

## Ursulas perform re-encryption

- Bob asks several Ursulas to re-encrypt the capsule so he can open it - He sents the capsule to Ursulas
- Each Ursula performs re-encryption on the capsule using the kfrag provided by Alice, obtaining this way a “capsule fragment”, or cfrag.
-  Let’s mock a network or transport layer by sampling threshold random kfrags, one for each required Ursula..


In [74]:
import random
kfrags = random.sample(kfrags,  # All kfrags from above (sent by Alice)
                       10)      # M - Threshold

capsule.set_correctness_keys(delegating=alices_public_key,
                             receiving=bobs_public_key,
                             verifying=alices_public_key)

cfrags = list()
for kfrag in kfrags:
    cfrag = pre.reencrypt(kfrag=kfrag, capsule=capsule)
    cfrags.append(cfrag)
cfrags

[CFrag:ceeade805a9d34c,
 CFrag:1f20c2653ffb240,
 CFrag:0d8f2266a93b5fc,
 CFrag:fe2a019a8d27820,
 CFrag:d16fdad8d9878ba,
 CFrag:92cfa579b9353d2,
 CFrag:f32a5c1b8859d35,
 CFrag:a2f887a5feed0b9,
 CFrag:406d54f5c31626e,
 CFrag:005c7b17defeb68]

## Bob activates and opens the capsule

- Bob collects the resulting cfrags from several Ursulas. Bob must gather at least threshold cfrags in order to activate the capsule.
- Bob attaches at least threshold cfrags to the capsule, which has to be prepared in advance with the necessary correctness keys.

In [75]:
capsule.set_correctness_keys(delegating=alices_public_key,
                             receiving=bobs_public_key,
                             verifying=alices_public_key)

for cfrag in cfrags:
    capsule.attach_cfrag(cfrag)

ciphertext = readFileIntoBytes(cipherTextFile)
cleartext = pre.decrypt(ciphertext=ciphertext, capsule=capsule, decrypting_key=bobs_private_key)
writeBytesIntoFile(decryptedTestFile, cleartext)